# Newton's Method Optimizer


## Setup

In [1]:
# Load necessary packages
import torch
import numpy as np
import math

## Step 1 - `newton()` function

The following `newton()` function demonstrates Newton’s method for minimization of functions using PyTorch and autograd functionality.  I mainly utilized two iterations to accomplish the task: one outer iteration which minimizes the objective function value with parameters upgrade, and one inner iteration which halves `Newton step` to ensure reduction in the objective function value. Before two iterations, the initial parameters are transformed into `torch` data types and whether there are finite objective or derivatives is checked. First, in the outer iteration, `Jacobian` and `Hessian` values are computed to derive `Newton step`. `Hessian` values are checked to be positive definite and if not, they are perturbed by adding multiples of the identity matrix. Second, in the inner iteration, parameters and objective function value are upgraded with `Newton step`. Whether `Newton step` reduces the objective function value is checked and if not, `Newton step` is halved. An error is raised if `Newton step` fails to reduce the objective function value with maximum step halving number. Third, back to the outer iteration, parameters, objective function value, iterations number and Jacobian values are upgraded. A specific condition for convergence is checked and an error is raised if failing to converge within maximum iterations number. In addition, whether `Hessian` value is positive definite at convergence is checked. Eventually, the `newton()` function returns a dictionary containing: the minimum objective function value, the parameters values, the iterations number and the gradient vector at the minimum.

In [2]:
def newton(theta, f, tol=1e-8, fscale=1.0, maxit=100, max_half=20):
    '''
    The function minimizes objective function value with Newton's method.
    Input arguments:
    theta: vector of initial values for optimization parameters
    f: objective function to minimize
    tol: convergence tolerance
    fscale: rough estimate of the magnitude of f at the optimum
    maxit: maximum number of Newton iterations to try before giving up
    max_half: maximum number of times a step should be halved before concluding failure
    Output:
    f: value of the objective function at the minimum
    theta: value of the parameters at the minimum
    iter: number of iterations taken to reach the minimum
    grad: gradient vector at the minimum
    '''
    # transform numpy array inputs to torch
    theta = torch.tensor(theta, dtype=torch.double)
    # compute objective and derivatives at initial theta 
    f_init = f(theta)
    j_init = torch.autograd.functional.jacobian(f, theta)
    h_init = torch.autograd.functional.hessian(f, theta)
    # raise an error if objective or derivatives are infinite
    if torch.isfinite(f_init) == False:
        raise ValueError('objective is not finite')
    if torch.all(torch.isfinite(j_init) == False)|torch.all(torch.isfinite(h_init) == False):
        raise ValueError('derivatives are not finite')
    # record values of objective function, theta and iterations number 
    all_f_i = [f(theta)]
    theta_i = theta
    iter_i = 0
    
    # begin minimization iteration (outer iteration)
    for i in range(maxit):
        # compute Jacobian and Hessian values
        j_i = torch.autograd.functional.jacobian(f, theta_i)  
        h_i = torch.autograd.functional.hessian(f, theta_i)
        # define function to check if Hessian is positive definite with Cholesky
        def pd(h):
            try:
                torch.linalg.cholesky(h)
                return True
            except torch.linalg.LinAlgError as e:
                return False  
        # test if Hessian is positive definite and make perturbed adjustments
        l = 1
        while pd(h_i) == False:
            # compute epsilon
            eps = 1e-8 * torch.max(torch.abs(h_i)) * l
            # add epsilon multiply the identity matrix
            h_i += eps * torch.eye(h_i.shape[0])
            # keep multiplying epsilon by 10
            l = l * 10
        # compute Newton step with Jacobian and Hessian values 
        step = - np.linalg.solve(h_i, j_i)
    
        # begin step halving iteration (inner iteration)
        for j in range(max_half):
            # upgrade theta and objective function values
            new_theta_i = theta_i + step
            new_f_i = f(new_theta_i)
            # check if Newton step reduces the objective function value
            if (new_f_i <= all_f_i[-1]):
                break
            # halve the Newton step    
            step /= 2
        # raise an error if Newton step fails to reduce the objective function value
        if (new_f_i > all_f_i[-1]):
            raise RuntimeError('fail to reduce the objective with maximum step halving number')
        
        # upgrade values of parameters, objective function, iterations number and Jacobian
        theta_i, f_i = new_theta_i, new_f_i
        all_f_i.append(f_i)
        iter_i = iter_i + 1   
        j_i = torch.autograd.functional.jacobian(f, theta_i) 
        # check condition for convergence
        if (max(abs(j_i))<(abs(f_i)+fscale)*tol):
            break
    # raise an error if failing to converge
    if (max(abs(j_i))>=(abs(f_i)+fscale)*tol):
        raise RuntimeError('fail to converge with maximum iterations')
    # raise an error if Hessian is not positive definite at convergence
    if (pd(h_i) == False) and (max(abs(j_i))<(abs(f_i)+fscale)*tol):
        raise ValueError('Hessian is not positive definite at convergence')
    # create a dictionary containing all outputs  
    dic = {'f':f_i, 'theta':theta_i, 'iter':iter_i, 'grad':j_i}
    
    return dic

## Step 2 - Minimization examples

For three minimization problems: Quadratic function, Rosenbrock's function and Poisson regression likelihood, I first implemented the function, and then utilized my `newton()` function to find the minima using at least 3 different theta starting values. In task 1, I already ensured that all the functions will take PyTorch `Tensors` as inputs and return a `Tensor`, so I do not need to set data types as `Tensor` again in the following minimization process.

### 1. Quadratic function

In [3]:
## Objective function implementation
## implement Quadratic function
def qua(x):
    return x[0]**2 - 2*x[0] + 2*x[1]**2 + x[1] + 3

In [4]:
## theta 1 minimization
newton([1,2], qua, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [5]:
## theta 2 minimization
newton([5,7], qua, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [6]:
## theta 3 minimization
newton([20,30], qua, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(1.8750, dtype=torch.float64),
 'theta': tensor([ 1.0000, -0.2500], dtype=torch.float64),
 'iter': 1,
 'grad': tensor([0., 0.], dtype=torch.float64)}

### 2. Rosenbrock's function

In [7]:
## Objective function implementation
## implement Rosenbrock's function
def rosen(x):
    return 10*(x[1] - x[0]**2)**2 + (1 - x[0])**2

In [8]:
## theta 1 minimization
newton([1,2], rosen, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(1.1703e-23, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 7,
 'grad': tensor([ 1.2867e-11, -3.0465e-12], dtype=torch.float64)}

In [9]:
## theta 2 minimization
newton([5,7], rosen, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(0., dtype=torch.float64),
 'theta': tensor([1., 1.], dtype=torch.float64),
 'iter': 17,
 'grad': tensor([0., 0.], dtype=torch.float64)}

In [10]:
## theta 3 minimization
newton([20,30], rosen, tol=1e-8, fscale=1.0, maxit=100, max_half=20)

{'f': tensor(9.0855e-25, dtype=torch.float64),
 'theta': tensor([1.0000, 1.0000], dtype=torch.float64),
 'iter': 37,
 'grad': tensor([ 1.1436e-11, -5.2491e-12], dtype=torch.float64)}

### 3. Poisson regression likelihood

In [11]:
## record original data 
x = [
   0.11, -0.06, -0.96, -0.48, -0.59, -0.42, -0.15,  1.14, 0.94, 
  -0.86, -0.08,  1.00, -2.01,  2.17, -0.20,  0.82, -0.13, 0.26, 
   0.22,  1.05
]

y = [4, 2, 4, 1, 1, 3, 4, 5, 7, 3, 5, 7, 0, 4, 2, 7, 3, 3, 2, 8]

In [12]:
## Objective function implementation
## implement log-likelihood function for Poisson regression 
def poisson(b):
    log_l = b[0] + b[1]*torch.tensor(x,dtype=torch.double)
    l = torch.exp(log_l)
    s = torch.sum(torch.mul(torch.tensor(y,dtype=torch.double),log_l)-l-torch.log(torch.tensor([math.factorial(m) for m in y])))
    return -s

In [13]:
## theta 1 minimization
newton([1,2], poisson, tol=1e-6, fscale=30.0, maxit=100, max_half=20)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 6,
 'grad': tensor([4.8569e-06, 6.8655e-06], dtype=torch.float64)}

In [14]:
## theta 2 minimization
newton([5,7], poisson, tol=1e-6, fscale=30.0, maxit=100, max_half=20)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 21,
 'grad': tensor([2.6478e-07, 3.7425e-07], dtype=torch.float64)}

In [15]:
## theta 3 minimization
newton([20,30], poisson, tol=1e-6, fscale=30.0, maxit=100, max_half=20)

{'f': tensor(37.8802, dtype=torch.float64),
 'theta': tensor([1.2089, 0.4279], dtype=torch.float64),
 'iter': 86,
 'grad': tensor([7.5907e-08, 1.0729e-07], dtype=torch.float64)}